In [ ]:
        data = pd.DataFrame({'y': y_test, 'X1': x_test[:,0], 'X2': x_test[:,1], 'X3': x_test[:,2], 'X4': x_test[:,3], 'd':treat_test, 'tr1': ctl_predict, 'tr2': cthl_predict})
        x1 = x_test[:,0]

        ate1_true = np.mean(y_test[np.where((x1 > 0) & (treat_test == 1))]) - np.mean(y_test[np.where((x1 > 0) & (treat_test == 0))])
        print(ate1_true == data['y'][(data['X1'] > 0) & (data['d'] == 1)].mean() - data['y'][(data['X1'] > 0) & (data['d'] == 0)].mean())
        ate1_test = np.mean(ctl_predict[np.where(x1 >= 0)])
        print(ate1_test == data['tr1'][data['X1'] > 0].mean())
        ate1_control = np.mean(cthl_predict[np.where(x1 >= 0)])
        print(ate1_control == data['tr2'][data['X1'] > 0].mean())
        print(ate1_true)
        ate0_true = np.mean(y_test[np.where((x1 < 0) & (treat_test == 1))]) - np.mean(y_test[np.where((x1 < 0) & (treat_test == 0))])
        print(ate0_true == data['y'][(data['X1'] < 0) & (data['d'] == 1)].mean() - data['y'][(data['X1'] < 0) & (data['d'] == 0)].mean())
        ate0_test = np.mean(ctl_predict[np.where(x1 < 0)])
        print(ate0_test == data['tr1'][data['X1'] < 0].mean())
        ate0_control = np.mean(cthl_predict[np.where(x1 < 0)])
        print(ate0_control == data['tr2'][data['X1'] < 0].mean())

In [ ]:
def mcreps():    
    ATE1 = []
    ATE0 = []

    reps = 10
    for i in range(reps):
        x_train, x_test, y_train, y_test, treat_train, treat_test = dgp3(1000)
        ate1, ate0, ctl_predict = MC_MSE_1(x_train, x_test, y_train, y_test, treat_train, treat_test, 0.5)
        ATE1.append(ate1)
        ATE0.append(ate0)

    print(np.sum(np.square(np.subtract(np.array(ATE1),1)))/reps)
    print(np.sum(np.square(np.add(np.array(ATE0), 1)))/reps)

In [ ]:
def mcreps_mse_all(reps, n, nomin_test):    
    ATE1 = []
    ATE0 = []
    MSEt = []
    true = np.ones(reps)
    
    for i in range(reps):
        x_train, x_test, y_train, y_test, treat_train, treat_test = dgp3(n)
        x1 = x_test[:,0]
        ctl_predict = MC_MSE_2(x_train, x_test, y_train, y_test, treat_train, treat_test, nomin_test)
        ATE1.append(np.mean(ctl_predict[np.where(x1 > 0)]))
        ATE0.append(np.mean(ctl_predict[np.where(x1 < 0)]))
        MSEt.append(np.sum(np.square(np.subtract(y_test[:,0],ctl_predict)))/n)

    mse_total = np.sum(np.array(MSEt))/reps
    mse1 = mse(np.array(ATE1),true)
    mse2 = mse(np.array(ATE0),true*-1)
    bias1 = np.sum(np.subtract(np.array(ATE1),true))/reps
    bias2 = np.sum(np.add(np.array(ATE0), true))/reps
    var1 = np.var(ATE1)
    var2 = np.var(ATE0)
    return mse1, mse2, bias1, bias2, var1, var2, mse_total